<a href="https://colab.research.google.com/github/fppfurtado/mvp-pos-ciencia-de-dados/blob/main/Projeto_MVP_Preditor_de_Pre%C3%A7os_de_T%C3%ADtulos_P%C3%BAblicos_Federais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Definição do Problema

## Objetivo
O presente projeto tem como objetivo a construção de um modelo de regressão linear para prever as taxas de juros ofertadas por títulos públicos federais - e, consequentemente, seus preços unitários - a partir dos valores da taxa de câmbio dolar/real (Ptax), do índice IBOVESPA e do CDS Brasil 5 anos.

## Títulos Públicos Federais
Títulos Públicos Federais são títulos de dívida emitidos pelo Governo Federal para financiar suas atvidades captando recursos de particulares em troca de pagamento de juros sobre um capital aplicado. Na prática, são **ativos de renda fixa** que possuem preço, taxa e vencimento definidos, e que podem ser adquiridos por investidores interessados.

Esses ativos possuem liquidez diária, podendo o investidor solicitar o resgate a qualquer tempo (durante o horário de funcionamento do mercado de títulos publicos), porém nesse caso o valor utilizado para o cálculo da rentabilidade será o **valor de mercado do título** no dia da solicitação de resgate, a chamada **Marcação a Mercado**, que pode fazer com que o investidor tenha ou lucro acima do esperado ou prejuízo no resgate antecipado.

O preço dos títulos públicos variam em função das respectivas taxas ofertadas. E as taxas variam diariamente, basicamente, em função da percepção de risco do mercado em relação à economia do país. Por isso, após alguma notícia negativa no cenário econômico, é comum observar fenômenos como "queda na bolsa", "alta no dolar" e "alta nos juros" pagos pelos títulos públicos federais.

## IBOVESPA
O Ibovespa é o principal indicador de desempenho das ações negociadas na B3 e reúne as empresas mais importantes do mercado de capitais brasileiro. Foi criado em 1968 e, ao longo desses 50 anos, consolidou-se como referência para investidores ao redor do mundo.

Reavaliado a cada quatro meses, o índice é resultado de uma carteira teórica de ativos. É composto pelas ações e units de companhias listadas na B3 que atendem aos critérios descritos na sua metodologia, correspondendo a cerca de 80% do número de negócios e do volume financeiro do nosso mercado de capitais.

Fonte: https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/ibovespa.htm

## PTax
Ptax é uma taxa de câmbio calculada durante o dia pelo Banco Central do Brasil. Consiste na média das taxas informadas pelas instituições financeiras durante 4 janelas do dia. É a taxa de referencia para o valor do dólar de D2 (em dois dias úteis). Normalmente, os contratos de derivativos de câmbio são liquidados com base na PTAX divulgada para o dia útil anterior.

Fonte: https://pt.wikipedia.org/wiki/Ptax

## CDS Brasil
CDS (credit default swap) é a troca de risco de crédito. O CDS é um contrato derivativo que funciona como um seguro para quem empresta dinheiro. Por levar em sua precificação o risco de inadimplência do tomador do empréstimo, o CDS é muito usado como termômetro para o risco de um país.

Fonte: https://inteligenciafinanceira.com.br/glossario/credit-default-swap-cds-risco-pais/

# 2. Coleta e Análise de Dados

## Importação de Pacotes e Módulos

In [4]:
# Importando pacotes
import pandas as pd
import warnings

## Configurações prévias

In [5]:
# Suprimindo warnings
warnings.filterwarnings('ignore')

## Carregamento dos Dados

Serão carregados os datasets dos dados históricos de cinco anos das quatro variáveis econômicas que serão tratadas posteriormente pelo modelo de regressão linear: 

+ taxas e preços de título públicos federais
+ taxa de câmbio PTax
+ índice IBOVESPA
+ CDS Brasil 5 anos.

In [6]:
# Carregando o histórico de preços de preços e taxas de títulos públicos
historico_titulos = pd.read_csv('https://github.com/fppfurtado/mvp-pos-ciencia-de-dados/raw/main/DADOS/historico_titulos.csv', decimal = ',')
# Carregando o histórico do índice IBOVESPA
historico_ibov = pd.read_csv('https://raw.githubusercontent.com/fppfurtado/mvp-pos-ciencia-de-dados/main/DADOS/historico_ibov.csv', decimal = ',')
# Carregando o histórico da taxa de câmbio PTax
historico_ptax = pd.read_csv('https://raw.githubusercontent.com/fppfurtado/mvp-pos-ciencia-de-dados/main/DADOS/historico_ptax.csv', decimal = ',')
# Carregando o histórico do CDS Brasil 5 Anos
historico_cds5y = pd.read_csv('https://raw.githubusercontent.com/fppfurtado/mvp-pos-ciencia-de-dados/main/DADOS/historico_csd5y.csv', decimal = ',')

Posteriormente, na seção de Pré-Processamento, será feita uma junção dos quatro datasets em um único dataset, apenas com os atributos relevantes para o modelo.

## Análise Exploratória

Para a exploração dos dados dos datasets, seguiremos o seguinte roteiro:

1. Análise das informações gerais sobre o dataset
2. Análise de amostras dos dados
3. Análise das principais estatísticas descritivas
4. Visualizações Unimodais dos atributos relevantes
5. Visualizações Multimodais dos atributos relevantes

### Análise Títulos Públicos

In [7]:
# Exibindo informações gerais sobre o dataset
historico_titulos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129257 entries, 0 to 129256
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Tipo Titulo        129257 non-null  object
 1   Data Vencimento    129257 non-null  object
 2   Data Base          129257 non-null  object
 3   Taxa Compra Manha  129257 non-null  object
 4   Taxa Venda Manha   129257 non-null  object
 5   PU Compra Manha    129257 non-null  object
 6   PU Venda Manha     129257 non-null  object
 7   PU Base Manha      129257 non-null  object
dtypes: object(8)
memory usage: 7.9+ MB


É possível notar que o dataset possui:
+ 08 colunas (atributos)
+ 129.257 linhas (instâncias)
+ 00 valores nulos (*missings*)

Nota-se também que os dados são todos do tipo "object", que basicamente significa que estão armazenados como *string* (texto). Isso dificultará manipulações e operações com esses dados, sendo necessário fazer a conversão dos atributos relevantes para o tipo adequado.

A falta de *missings* é explicada pela fato de o dataset não possuir dados oriundos diretamente de entradas de usuário (ex.: formulários), mas apenas dados computados automaticamente pelo sistema de negociação de títulos públicos.

In [11]:
# convertendo atributos relevantes para os tipos adequados
historico_titulos = historico_titulos.astype(
    {
      'Data Vencimento': 'datetime64',
      'Data Base': 'datetime64',
      'Taxa Compra Manha': 'float',
      'Taxa Venda Manha': 'float',
      'PU Compra Manha': 'float',
      'PU Venda Manha': 'float',
      'PU Base Manha': 'float',
    }
)

In [12]:
# exibindo linhas de fronteiras (início e fim) do dataset ordenadas pelo atributo 'Data Base'
historico_titulos.sort_values('Data Base', ascending = False)

,Tipo Titulo,Data Vencimento,Data Base,Taxa Compra Manha,Taxa Venda Manha,PU Compra Manha,PU Venda Manha,PU Base Manha
4799,Tesouro Renda+ Aposentadoria Extra,2069-12-15,2023-04-10,6.17,6.29,484.85,466.01,466.01
4775,Tesouro IPCA+ com Juros Semestrais,2055-05-15,2023-04-10,6.16,6.28,4110.76,4043.73,4043.73
4783,Tesouro Renda+ Aposentadoria Extra,2054-12-15,2023-04-10,6.22,6.34,1175.76,1149.36,1149.36
4782,Tesouro Renda+ Aposentadoria Extra,2074-12-15,2023-04-10,6.14,6.26,363.80,347.69,347.69
4780,Tesouro Prefixado,2026-01-01,2023-04-10,11.96,12.08,734.59,732.12,732.12
...,...,...,...,...,...,...,...,...
7040,Tesouro IGPM+ com Juros Semestrais,2011-03-01,2004-12-31,8.33,8.39,1647.67,1643.07,1641.39
7041,Tesouro IGPM+ com Juros Semestrais,2017-07-01,2004-12-31,8.26,8.34,1500.78,1491.30,1543.01
7042,Tesouro IGPM+ com Juros Semestrais,2021-04-01,2004-12-31,8.25,8.33,1482.71,1472.08,1470.58
7043,Tesouro IGPM+ com Juros Semestrais,2031-01-01,2004-12-31,8.24,8.34,2508.78,2484.40,2586.83


É possível verificar pela amostra acima que o dataset apresenta dados coerentes com o significado de seus atributos, bem como coerência na ordenação temporal pelo atributo "Data Base".

### Análise IBOVESPA

### Análise PTax

### Análise CDS 5Y